In [2]:
import pandas as pd
import ast
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [3]:
review_df = pd.read_csv('australian_user_reviews.csv')
items_df = pd.read_csv('items.csv')
games_df = pd.read_csv('output_steam_games.csv')

C:\Users\USER\AppData\Local\Temp\ipykernel_20208\3428498735.py:3: DtypeWarning: Columns (0,1,2,3,4,5,6,7,9,10,11,14,15,17) have mixed types. Specify dtype option on import or set low_memory=False.
  games_df = pd.read_csv('output_steam_games.csv')


In [4]:
review_df

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [5]:
review_df.drop(columns='user_url', inplace=True)

In [6]:
review_df

,user_id,reviews
0,76561197970982479,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...
25794,76561198306599751,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [14]:
# Lista para almacenar los datos extraídos
new_data = []

# Iterar a través de las filas del DataFrame original
for index, row in review_df.iterrows():
    user_id = row['user_id']
    reviews_str = row['reviews']

    # Convertir la cadena de reseñas en una lista de diccionarios
    reviews_list = ast.literal_eval(reviews_str)

    # Iterar a través de las reseñas del usuario
    for review in reviews_list:
        item_id = review['item_id']
        recommend = review['recommend']
        review_text = review['review']

        # Agregar los datos extraídos a la lista
        new_data.append({'user_id': user_id, 'item_id': item_id, 'recommend': recommend, 'review': review_text})

# Crear un nuevo DataFrame con los datos extraídos
new_review_df = pd.DataFrame(new_data)

In [15]:
new_review_df

,user_id,item_id,recommend,review
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,22200,True,It's unique and worth a playthrough.
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,251610,True,I know what you think when you see this title ...
4,js41637,227300,True,For a simple (it's actually not all that simpl...
...,...,...,...,...
59300,76561198312638244,70,True,a must have classic from steam definitely wort...
59301,76561198312638244,362890,True,this game is a perfect remake of the original ...
59302,LydiaMorley,273110,True,had so much fun plaing this and collecting res...
59303,LydiaMorley,730,True,:D


In [16]:
# Crear el objeto para análisis de sentimiento VADER
sia = SentimentIntensityAnalyzer()

# Función para asignar el valor de sentimiento
def get_sentiment(row):
    try:
        sentiment_scores = sia.polarity_scores(row['review'])
        compound_score = sentiment_scores['compound']
        
        if compound_score >= 0.05:
            return 2  # Positivo
        elif compound_score > -0.05 and compound_score < 0.05:
            return 1  # Neutral
        else:
            return 0  # Negativo
    except:
        return 1  # Valor predeterminado si no se puede analizar el sentimiento

# Aplicar la función y crear la nueva columna 'sentiment_analysis'
new_review_df['sentiment_analysis'] = new_review_df.apply(get_sentiment, axis=1)


In [17]:
new_review_df.drop(columns=['review'], inplace=True)

In [18]:
new_review_df

,user_id,item_id,recommend,sentiment_analysis
0,76561197970982479,1250,True,2
1,76561197970982479,22200,True,2
2,76561197970982479,43110,True,2
3,js41637,251610,True,2
4,js41637,227300,True,2
...,...,...,...,...
59300,76561198312638244,70,True,2
59301,76561198312638244,362890,True,2
59302,LydiaMorley,273110,True,2
59303,LydiaMorley,730,True,2


In [4]:
items_df

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [42]:
# Lista para almacenar los datos extraídos
new_data = []

# Iterar a través de las filas del DataFrame original
for index, row in items_df.iterrows():
    user_id = row['user_id']
    item_str = row['items']
    items_count = row['items_count']

    # Convertir la cadena de reseñas en una lista de diccionarios
    item_list = ast.literal_eval(item_str)

    # Iterar a través de las reseñas del usuario
    for item in item_list:
        item_id = item['item_id']
        item_name = item['item_name']
        playtime = item['playtime_forever']

        # Agregar los datos extraídos a la lista
        new_data.append({'user_id': user_id, 'items_count': items_count, 'item_id': item_id, 'item_name': item_name, 'playtime_forever': playtime})

# Crear un nuevo DataFrame con los datos extraídos
new_item_df = pd.DataFrame(new_data)

In [43]:
new_item_df

,user_id,items_count,item_id,item_name,playtime_forever
0,76561197970982479,277,10,Counter-Strike,6
1,76561197970982479,277,20,Team Fortress Classic,0
2,76561197970982479,277,30,Day of Defeat,7
3,76561197970982479,277,40,Deathmatch Classic,0
4,76561197970982479,277,50,Half-Life: Opposing Force,0
...,...,...,...,...,...
5153204,76561198329548331,7,346330,BrainBread 2,0
5153205,76561198329548331,7,373330,All Is Dust,0
5153206,76561198329548331,7,388490,One Way To Die: Steam Edition,3
5153207,76561198329548331,7,521570,You Have 10 Seconds 2,4


In [3]:
games_df = pd.read_csv('output_steam_games.csv')

C:\Users\USER\AppData\Local\Temp\ipykernel_6428\3277710189.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,9,10,11,14,15,17) have mixed types. Specify dtype option on import or set low_memory=False.
  games_df = pd.read_csv('output_steam_games.csv')


In [4]:
games_df

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer,user_id,steam_id,items,items_count
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76561197970982479,7.656120e+16,"[{'item_id': '10', 'item_name': 'Counter-Strik...",277.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,js41637,7.656120e+16,"[{'item_id': '10', 'item_name': 'Counter-Strik...",888.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,evcentric,7.656120e+16,"[{'item_id': '1200', 'item_name': 'Red Orchest...",137.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Riot-Punch,7.656120e+16,"[{'item_id': '10', 'item_name': 'Counter-Strik...",328.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,doctr,7.656120e+16,"[{'item_id': '300', 'item_name': 'Day of Defea...",541.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"['Casual', 'Indie', 'Simulation', 'Strategy']",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"['Strategy', 'Indie', 'Casual', 'Simulation']",http://steamcommunity.com/app/773640/reviews/?...,1.49,"['Single-player', 'Steam Achievements']",1.99,False,773640.0,NaN,"Nikita ""Ghost_RUS""",NaN,NaN,NaN,NaN
120441,Sacada,"['Casual', 'Indie', 'Strategy']",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"['Strategy', 'Indie', 'Casual']",http://steamcommunity.com/app/733530/reviews/?...,4.24,"['Single-player', 'Steam Achievements', 'Steam...",4.99,False,733530.0,NaN,Sacada,NaN,NaN,NaN,NaN
120442,Laush Studio,"['Indie', 'Racing', 'Simulation']",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"['Indie', 'Simulation', 'Racing']",http://steamcommunity.com/app/610660/reviews/?...,1.39,"['Single-player', 'Steam Achievements', 'Steam...",1.99,False,610660.0,NaN,Laush Dmitriy Sergeevich,NaN,NaN,NaN,NaN
120443,SIXNAILS,"['Casual', 'Indie']",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"['Indie', 'Casual', 'Puzzle', 'Singleplayer', ...",http://steamcommunity.com/app/658870/reviews/?...,NaN,"['Single-player', 'Steam Achievements', 'Steam...",4.99,False,658870.0,NaN,"xropi,stev3ns",NaN,NaN,NaN,NaN


In [5]:
games_df.dropna(subset=['app_name'], inplace=True)

In [6]:
games_df

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer,user_id,steam_id,items,items_count
88310,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,4.49,['Single-player'],4.99,False,761140.0,NaN,Kotoshiro,NaN,NaN,NaN,NaN
88311,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",http://steamcommunity.com/app/643980/reviews/?...,NaN,"['Single-player', 'Multi-player', 'Online Mult...",Free To Play,False,643980.0,NaN,Secret Level SRL,NaN,NaN,NaN,NaN
88312,Poolians.com,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"['Free to Play', 'Simulation', 'Sports', 'Casu...",http://steamcommunity.com/app/670290/reviews/?...,NaN,"['Single-player', 'Multi-player', 'Online Mult...",Free to Play,False,670290.0,NaN,Poolians.com,NaN,NaN,NaN,NaN
88313,彼岸领域,"['Action', 'Adventure', 'Casual']",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"['Action', 'Adventure', 'Casual']",http://steamcommunity.com/app/767400/reviews/?...,0.83,['Single-player'],0.99,False,767400.0,NaN,彼岸领域,NaN,NaN,NaN,NaN
88314,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"['Action', 'Indie', 'Casual', 'Sports']",http://steamcommunity.com/app/773570/reviews/?...,1.79,"['Single-player', 'Full controller support', '...",2.99,False,773570.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"['Casual', 'Indie', 'Simulation', 'Strategy']",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"['Strategy', 'Indie', 'Casual', 'Simulation']",http://steamcommunity.com/app/773640/reviews/?...,1.49,"['Single-player', 'Steam Achievements']",1.99,False,773640.0,NaN,"Nikita ""Ghost_RUS""",NaN,NaN,NaN,NaN
120441,Sacada,"['Casual', 'Indie', 'Strategy']",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"['Strategy', 'Indie', 'Casual']",http://steamcommunity.com/app/733530/reviews/?...,4.24,"['Single-player', 'Steam Achievements', 'Steam...",4.99,False,733530.0,NaN,Sacada,NaN,NaN,NaN,NaN
120442,Laush Studio,"['Indie', 'Racing', 'Simulation']",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"['Indie', 'Simulation', 'Racing']",http://steamcommunity.com/app/610660/reviews/?...,1.39,"['Single-player', 'Steam Achievements', 'Steam...",1.99,False,610660.0,NaN,Laush Dmitriy Sergeevich,NaN,NaN,NaN,NaN
120443,SIXNAILS,"['Casual', 'Indie']",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"['Indie', 'Casual', 'Puzzle', 'Singleplayer', ...",http://steamcommunity.com/app/658870/reviews/?...,NaN,"['Single-player', 'Steam Achievements', 'Steam...",4.99,False,658870.0,NaN,"xropi,stev3ns",NaN,NaN,NaN,NaN


In [7]:
games_df.drop(columns= ['user_id', 'steam_id', 'items', 'items_count', 'reviews_url', 'url' ], inplace= True)

In [8]:
games_df

,publisher,genres,app_name,title,release_date,tags,discount_price,specs,price,early_access,id,metascore,developer
88310,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",4.49,['Single-player'],4.99,False,761140.0,NaN,Kotoshiro
88311,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,Ironbound,2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",NaN,"['Single-player', 'Multi-player', 'Online Mult...",Free To Play,False,643980.0,NaN,Secret Level SRL
88312,Poolians.com,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,Real Pool 3D - Poolians,2017-07-24,"['Free to Play', 'Simulation', 'Sports', 'Casu...",NaN,"['Single-player', 'Multi-player', 'Online Mult...",Free to Play,False,670290.0,NaN,Poolians.com
88313,彼岸领域,"['Action', 'Adventure', 'Casual']",弹炸人2222,弹炸人2222,2017-12-07,"['Action', 'Adventure', 'Casual']",0.83,['Single-player'],0.99,False,767400.0,NaN,彼岸领域
88314,NaN,NaN,Log Challenge,NaN,NaN,"['Action', 'Indie', 'Casual', 'Sports']",1.79,"['Single-player', 'Full controller support', '...",2.99,False,773570.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"['Casual', 'Indie', 'Simulation', 'Strategy']",Colony On Mars,Colony On Mars,2018-01-04,"['Strategy', 'Indie', 'Casual', 'Simulation']",1.49,"['Single-player', 'Steam Achievements']",1.99,False,773640.0,NaN,"Nikita ""Ghost_RUS"""
120441,Sacada,"['Casual', 'Indie', 'Strategy']",LOGistICAL: South Africa,LOGistICAL: South Africa,2018-01-04,"['Strategy', 'Indie', 'Casual']",4.24,"['Single-player', 'Steam Achievements', 'Steam...",4.99,False,733530.0,NaN,Sacada
120442,Laush Studio,"['Indie', 'Racing', 'Simulation']",Russian Roads,Russian Roads,2018-01-04,"['Indie', 'Simulation', 'Racing']",1.39,"['Single-player', 'Steam Achievements', 'Steam...",1.99,False,610660.0,NaN,Laush Dmitriy Sergeevich
120443,SIXNAILS,"['Casual', 'Indie']",EXIT 2 - Directions,EXIT 2 - Directions,2017-09-02,"['Indie', 'Casual', 'Puzzle', 'Singleplayer', ...",NaN,"['Single-player', 'Steam Achievements', 'Steam...",4.99,False,658870.0,NaN,"xropi,stev3ns"


In [14]:
games_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32133 entries, 88310 to 120444
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   publisher       24073 non-null  object 
 1   genres          28851 non-null  object 
 2   app_name        32133 non-null  object 
 3   title           30085 non-null  object 
 4   release_date    30067 non-null  object 
 5   tags            31971 non-null  object 
 6   discount_price  32133 non-null  float64
 7   specs           31464 non-null  object 
 8   price           30757 non-null  object 
 9   early_access    32133 non-null  object 
 10  id              32132 non-null  float64
 11  metascore       2607 non-null   float64
 12  developer       28836 non-null  object 
dtypes: float64(3), object(10)
memory usage: 3.4+ MB


In [13]:
games_df.isnull().sum()

publisher          8060
genres             3282
app_name              0
title              2048
release_date       2066
tags                162
discount_price        0
specs               669
price              1376
early_access          0
id                    1
metascore         29526
developer          3297
dtype: int64

In [11]:
games_df['discount_price'].fillna(0, inplace=True)